
# Trying out second form of PDE

Implemented in modesolver as leakymode2 vecmodesystem2.  Debugging here.

In [4]:
import numpy as np
import ngsolve as ng

from fiberamp.fiber.microstruct.bragg import Bragg
from ngsolve.webgui import Draw
from ngsolve import CF

from pyeigfeast import NGvecs, SpectralProjNGGeneral

In [118]:
n_air = 1.00027717
n_glass = 1.4388164768221814
ts = [15*2.7183333333333333e-6, 15*2/3*1e-6, 15*1e-6]
ns = [lambda x: n_air, lambda x: n_glass, lambda x: n_glass]
mats = ['air', 'glass', 'Outer']
maxhs = [.1, .015, .025]
scale = 15e-6
i=0
wls = np.linspace(1.4, 2, 301) * 1e-6

A = Bragg(ts=ts, scale=scale, maxhs=maxhs, ns=ns, mats=mats, wl=wls[i])


ModeSolver: Checking if mesh has required regions
Mesh has  18030  elements,  9134  points,  and  27163  edges.


In [119]:
p = 3
alpha = .2

A.mesh.SetPML(ng.pml.Radial(rad=A.R, alpha=alpha*1j, origin=(0, 0)),'Outer')

k2 = A.k ** 2 * A.L**2 * A.index * A.index

dr = ng.ds(definedon=~A.mesh.Materials('Outer'), skeleton=1)
n = ng.specialcf.normal(2)

X = ng.HCurl(A.mesh, order=p+1-max(1-p, 0), type1=True,
             dirichlet='OuterCircle', dgjumps=True, complex=True)
Y = ng.H1(A.mesh, order=p+1, dgjumps=True, complex=True)
XY = ng.FESpace([X,Y])

(E, phi), (v, psi) = XY.TnT()

AA = ng.BilinearForm(XY)
AA += (ng.curl(E) * ng.curl(v) - k2 * E * v) * ng.dx
AA += ng.grad(phi) * v * ng.dx
AA += ng.grad(phi) * ng.grad(psi) * ng.dx
AA += -k2 * phi * psi * ng.dx
AA += -psi * (ng.grad(phi.Other()) - ng.grad(phi)) * n * dr

B= ng.BilinearForm(XY)
B += -E * v * ng.dx
B += -E * ng.grad(psi) * ng.dx
B += psi * (E.Other() - E) * n * dr

MM = ng.BilinearForm(XY)
MM += (E * v + phi * psi) * ng.dx

with ng.TaskManager():
    try:
        AA.Assemble()
        B.Assemble()
        MM.Assemble()

    except Exception:
        print('*** Trying again with larger heap')
        ng.SetHeapSize(int(1e9))
        AA.Assemble()
        B.Assemble()
        MM.Assemble()


In [120]:
exact_scaled_betas = np.load('/home/pv/local/convergence/bragg_fiber/wavelength/N0/exact_scaled_betas.npy').conjugate()
centers = exact_scaled_betas **2

In [121]:
ctr = centers[i]
rad = .01
npts = 2

P = SpectralProjNGGeneral(XY, AA.mat, B.mat, radius=rad, center=ctr,
                          npts=npts)


SpectralProj: Setting shifted trapezoidal rule quadrature on circular contour
SpectralProj: Radius=0.01, Center=4533.69+0.0126895j

SpectralProjNG: Computing resolvents along the contour using
                inverse type umfpack
SpectralProjNG:   Factorizing at z = +4533.691 +0.023j
SpectralProjNG:   Factorizing at z = +4533.691 +0.003j


In [122]:
nspan = 2

E_phi = NGvecs(XY, nspan, M=MM.mat)
E_phi_l = E_phi.create()
E_phi.setrandom(seed=1)
E_phi_l.setrandom(seed=1)

In [123]:
beta_sqrs, E_phi, history, E_phi_l = P.feast(E_phi, Yl=E_phi_l, niterations=5, nrestarts=0,
                                             hermitian=False)

betas = np.sqrt(beta_sqrs, dtype=complex)/A.L
Zsqrs = A.sqrZfrom(betas)


=========== Starting FEAST iterations ===========
Trying with 2 vectors:

 ITERATION 1 with 2 vectors and 2 left vectors
   During kernel cleaning abs(g):
   [ 2.4 20.5]
   Real part of computed eigenvalues:
   [4533.69231318 4533.69254513]
   Imag part of computed eigenvalues:
   [0.0114795  0.01143067]
   Relative Hausdorff distance from prior iterate: 2.206e+96

 ITERATION 2 with 2 vectors and 2 left vectors
   During kernel cleaning abs(g):
   [0.1 0.1]
   Real part of computed eigenvalues:
   [4533.69248144 4533.69249693]
   Imag part of computed eigenvalues:
   [0.01136122 0.0114473 ]
   Relative Hausdorff distance from prior iterate: 4.115e-08

 ITERATION 3 with 2 vectors and 2 left vectors
   During kernel cleaning abs(g):
   [ 9.2 10.1]
   Real part of computed eigenvalues:
   [4533.69248144 4533.69249693]
   Imag part of computed eigenvalues:
   [0.0113612  0.01144731]
   Relative Hausdorff distance from prior iterate: 3.574e-12


***FEAST converged with:
	Relative Hausdorff

In [126]:
for e in E_phi:
    Draw(-1e3*e.components[0].real[0],A.mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

In [127]:
for e in E_phi:
    Draw(1e2*e.components[1],A.mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

# Automated version hopefully

In [4]:
alpha=1
nspan = 4
p = 1
center = centers[i]
radius = .1
npts = 2

beta, _, E_phis, _ = A.leakyvecmodes2(ctr=center,
                                       rad=radius,
                                       alpha=None,
                                       d=None,
                                       nspan=nspan,
                                       npts=npts,
                                       p=p,
                                       niterations=4,
                                       nrestarts=0,
                                       stop_tol=1e-9,)

No PML.
Using FEAST to search for vector leaky modes in
circle of radius 0.1 centered at  (4533.691333203114+0.012689514720893767j)
assuming not more than 4 modes in this interval
System size: 76571  x  76571   Inverse type: umfpack

SpectralProj: Setting shifted trapezoidal rule quadrature on circular contour
SpectralProj: Radius=0.1, Center=4533.69+0.0126895j
SpectralProjNGR:   Making resolvent at z = +4533.691 +0.113j
SpectralProjNGR:   Making resolvent at z = +4533.691 -0.087j

=========== Starting FEAST iterations ===========
Trying with 4 vectors:

 ITERATION 1 with 4 vectors and 4 left vectors
   During kernel cleaning abs(g):
   [22367.5  2864.8   120.     39.9]
   Real part of computed eigenvalues:
   [ -54.34619437  758.40017219 4533.77454804 4557.93593358]
   Imag part of computed eigenvalues:
   [ -68.03570406 -647.64355426  -80.43214549   35.20012003]
   Relative Hausdorff distance from prior iterate: 2.206e+96

 ITERATION 2 with 4 vectors and 4 left vectors
   During kern